In [1]:
!rm -rf colab_package.zip database requirements.txt src

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import zipfile

from google.colab import files
uploaded = files.upload()

with zipfile.ZipFile('colab_package.zip', "r") as zf:
    zf.extractall(".")
    
print("✅ Package extracted")

✅ Package extracted


In [ ]:
!ls
# !ls -R

In [ ]:
# 2. install requirements with GPU support
%pip install -q -r requirements.txt
!python -m spacy download en_core_web_sm

# Enable GPU for PyTorch
import torch
import tensorflow as tf

if torch.cuda.is_available():
    print(f"✅ GPU Enabled: {torch.cuda.get_device_name()}")
    print(f"TensorFlow GPU: {len(tf.config.list_physical_devices('GPU'))}")
else:
    print("⚠️ No GPU (still faster than laptop)")

Note: you may need to restart the kernel to use updated packages.
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠️ No GPU (still faster than laptop)


In [ ]:
# 3. set environment to use GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from dotenv import load_dotenv
load_dotenv()

!pip install -U spacy[cuda11x,transformers]

# Set memory growth to avoid OOM
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

print("✅ Environment configured")

✅ Environment configured


In [ ]:
# Fix CuPy
!pip uninstall -y cupy cupy-cuda10x cupy-cuda11x cupy-cuda12x cupy-cuda-runtime
!pip install cupy-cuda11x

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
checkpoint_dir = "/content/drive/MyDrive/ColabCheckpoints"
# Create the directory if it doesn't exist
os.makedirs(checkpoint_dir, exist_ok=True)

In [ ]:
!PYTHONPATH=. python -m src.processing.main

In [ ]:
# misc: code to give insight on the checkpoint
from google.colab import drive
drive.mount('/content/drive')

def analyze_all_checkpoints(checkpoint_dir="/content/drive/MyDrive/ColabCheckpoints"):
    from pathlib import Path
    import pickle
    
    checkpoint_path = Path(checkpoint_dir)
    checkpoints = list(checkpoint_path.glob("*_checkpoint.pkl"))
    
    stage_counts = {}
    firms_by_stage = {}
    
    for cp_file in checkpoints:
        with open(cp_file, 'rb') as f:
            cp_data = pickle.load(f)
            stage = cp_data.get('stage', 'Unknown')
            firm = cp_data.get('firm_name', 'Unknown')
            
            stage_counts[stage] = stage_counts.get(stage, 0) + 1
            if stage not in firms_by_stage:
                firms_by_stage[stage] = []
            firms_by_stage[stage].append(firm)
    
    print("Checkpoint Analysis:")
    print(f"Total checkpoints: {len(checkpoints)}")
    print("\nBreakdown by stage:")
    for stage, count in stage_counts.items():
        print(f"  - {stage}: {count} firms")
        
    return firms_by_stage

# Run this to confirm all are at topic_modeling
firms_by_stage = analyze_all_checkpoints()